In [4]:
import pandas as pd 
import numpy as np
import lightgbm as lgb

data = pd.read_csv("C:/Users/lifebloom/.kaggle/competitions/home-credit-default-risk/application_train.csv")
test = pd.read_csv("C:/Users/lifebloom/.kaggle/competitions/home-credit-default-risk/application_test.csv")

In [5]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
param_test ={'num_leaves': sp_randint(6, 50), 
             'min_child_weight': sp_randint(1, 500), 
             'colsample_bytree': sp_uniform(loc=0.6, scale=0.4), 
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

In [6]:
n_HP_points_to_test = 20
from sklearn.model_selection import RandomizedSearchCV
clf = lgb.LGBMClassifier(max_depth=-1, is_unbalance=True, random_state=314, silent=True, metric='None', n_jobs=5)
gs = RandomizedSearchCV(
    estimator=clf, param_distributions=param_test, 
    n_iter=n_HP_points_to_test,
    scoring='roc_auc',
    cv=5,
    refit=True,
    random_state=314,
    verbose=True)

In [ ]:
gs.fit(data.filter(regex=r'^EXT_SOURCE_.', axis=1), data['TARGET'])
print('Best score reached: {} with params: {} '.format(gs.best_score_, gs.best_params_))

Fitting 5 folds for each of 20 candidates, totalling 100 fits


In [ ]:
for i in np.argsort(gs.cv_results_['mean_test_score'])[-5:]:
    print('{1:.4f}+-{3:.4f} {2:.4f}   :  {0}'.format(gs.cv_results_['params'][i], 
                                    gs.cv_results_['mean_test_score'][i], 
                                    gs.cv_results_['mean_train_score'][i],
                                    gs.cv_results_['std_test_score'][i]))

In [ ]:
gs.cv_results_.keys()

In [ ]:
probabilities = gs.best_estimator_.predict_proba(test.filter(regex=r'^EXT_SOURCE_.', axis=1))
submission = pd.DataFrame({
    'SK_ID_CURR': test['SK_ID_CURR'],
    'TARGET':     [ row[1] for row in probabilities]
})
submission.to_csv("submission.csv", index=False)